In [2]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

tags = {
    "loss": "Loss_MB/train_phase/train_stream/Task000",
    "final_accuracy": "Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000",
}

def load_event_file(experiment_directory: str) -> EventAccumulator:
    """
    Loads the event file from a given experiment directory.
    """
    ea: EventAccumulator = EventAccumulator(experiment_directory)
    ea.Reload()
    return ea

def final_metrics(ea: EventAccumulator) -> dict:
    """
    Returns the final metrics from the event file.
    """
    result = {}
    for tag_name, tag in tags.items():
        if tag in ea.Tags()['scalars']:
            result[tag_name] = ea.Scalars(tag)[-1].value
    return result



In [3]:
experiment_logs = "experiment_logs"

records = []

# Loop over each directory in a directory
for experiment in tqdm(os.listdir(os.path.join(experiment_logs))):
    i, dataset, arch, strategy = experiment.split("_")

    record = {
        "dataset": dataset,
        "architecture": arch,
        "strategy": strategy,
        "id": i,
    }

    try:
        ea = load_event_file(os.path.join(experiment_logs, experiment))
        record.update(final_metrics(ea))
    except KeyError as e:
        print(f"Could not load required metric {e} in '{experiment}'")

    records.append(record)

df = pd.DataFrame.from_dict(records)
df

  0%|          | 0/47 [00:00<?, ?it/s]

,dataset,architecture,strategy,id,loss,final_accuracy
0,splitEmbeddedCIFAR100,AE,cumulative,0049,1.520590,0.323200
1,splitEmbeddedCIFAR100,AE,finetuning,0047,0.334368,0.075200
2,splitCORe50,VAE,cumulative,0021,0.321811,0.360358
3,splitCIFAR10,AE,cumulative,0015,0.286854,0.872100
4,splitCIFAR10,AE,taskOracle,0032,0.422142,0.937500
5,splitEmbeddedCIFAR100,AE,taskInference,0053,2.840612,0.219400
6,splitCIFAR100,VAE,taskOracle,0037,1.664222,0.591000
7,splitCIFAR100,AE,taskOracle,0033,1.803951,0.547500
8,splitFMNIST,VAE,taskInference,0043,0.252686,0.992700
9,splitFMNIST,VAE,taskInference,0006,NaN,NaN


In [4]:
df.dropna().sort_values(by='id').groupby(['dataset', 'architecture', 'strategy']).max().pivot_table(index=['architecture', 'strategy'], columns=['dataset'], values='final_accuracy').to_csv('accuracy.csv')

In [5]:
df.to_csv("results/results.csv")

In [12]:

for strategy in ["cumulative", "taskOracle", "finetuning", "taskInference"]:
    for arch in ["AE", "VAE"]:
        for dataset in [
            "splitFMNIST",
            "splitCIFAR10",
            "splitCIFAR100",
            "splitCORe50",
            "splitEmbeddedCIFAR100",
            "splitEmbeddedCORe50"]:
            try:
                result = df[df['dataset'] == dataset]
                result = result[result['strategy'] == strategy]
                result = result[result['architecture'] == arch]
                accuracy = result.sort_values(by='id').max()["final_accuracy"]
                # accuracy = result.sort_values("id")[-1]["accuracy"]
                # display(result.sort_values(by='id'))
                print(f"{accuracy:0.4f}", end="\t")
            except:
                continue
        print()

0.9151	0.8721	0.6167	0.3560	0.3232	nan	
0.8989	0.8725	0.6106	0.3604	0.3080	nan	
0.9938	0.9375	0.5475	0.4322	0.3688	nan	
0.9936	0.9382	0.5910	0.4032	0.2850	nan	
0.1995	0.1939	0.0899	0.0745	0.0752	nan	
0.1995	0.1945	0.0891	0.0751	0.0747	nan	
0.9597	0.7123	0.2351	0.0553	0.2194	nan	
0.9927	0.7295	0.3091	0.0531	0.2157	nan	
